In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow import keras, config
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("GPUs Available: ", len(config.list_physical_devices('GPU')))

In [ ]:
def show_image(img, width):
    plt.figure(figsize=(30,5))
    plt.xlim(0, width)
    plt.imshow(img)
    plt.show()

In [ ]:
%load_ext autoreload
%autoreload 2
from image_process.image_preprocessors import shift_randomly_binary_labels
from data_filters import min_n_letters, take_percent
from keras_generators.image_generator import Image_Generator_RAW
# Global settings
TEST_SPLIT_SIZE = 0.80
VALIDATION_SPLIT_SIZE = 0.90
IMAGE_TARGET_SIZE = (5, 1400, 1)
BATCH_SIZE =  128
LETTER_END_POSITION = "P1"
ADD_NOISE_RANDOMLY = [0, 30]
ADD_SIGNAL_INDENT_RANDOMLY = [12860, 12860]
IMAGE_PREPOCESSORS = [
    {"func": shift_randomly_binary_labels, "params" : [-200, -100]},
]

In [ ]:
from Image_Generator_helpers import  DataSets, set_paths, global_path
from data_filters import min_n_letters_raw
set_obj = DataSets(set_paths, global_path, [min_n_letters_raw(3)])
set_obj.csv_files

# (data_frame["Tempo"].values.astype(np.float) / data_frame["Tempo Diff"].values.astype(np.float)) / 100

def get_velocity_labels(random_items):
    return [(item.csv_row["Tempo"].values.astype(np.float64)[0] / item.csv_row["Tempo Diff"].values.astype(np.float64)[0]) / 100 for item in random_items]


In [ ]:

Train_Generator_RAW = Image_Generator_RAW(
    image_amount=BATCH_SIZE * 1000,
    set_obj= set_obj,
    FFT_JUMP=64,
    batch_size=BATCH_SIZE,
    image_target_size=IMAGE_TARGET_SIZE,
    image_prepocessors=IMAGE_PREPOCESSORS,
    noise_range=ADD_NOISE_RANDOMLY,
    random_signal_indent=ADD_SIGNAL_INDENT_RANDOMLY,
    label_func = get_velocity_labels
    )

# Validation_Generator_RAW = Image_Generator_RAW(
#     image_amount=BATCH_SIZE * 1000,
#     set_obj= set_obj,
#     FFT_JUMP=64,
#     batch_size=BATCH_SIZE,
#     image_target_size=IMAGE_TARGET_SIZE,
#     image_prepocessors=IMAGE_PREPOCESSORS,
#     noise_range=ADD_NOISE_RANDOMLY,
#     random_signal_indent=ADD_SIGNAL_INDENT_RANDOMLY,
#     label_func = get_velocity_labels
#     )

# (t,l) = Train_Generator_RAW.__getitem__(0)


# for idx,img in enumerate(t):
#     print("label: ")
#     print(l[idx])
#     show_image(img, 200)
    

In [ ]:
def conv_model_velocity(input_layer):
    x = keras.layers.Cropping2D(cropping=((0, 0), (0,200)), data_format=None)(input_layer)

    x = Conv2D(90,(1,7), padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,1),padding="same")(x)

    x = Conv2D(90,(1,7),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(1,5),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Flatten()(x)

    return x

In [ ]:
from tensorflow.keras import layers

input_layer = Input(shape=IMAGE_TARGET_SIZE)
conv_model_position_flattened = conv_model_velocity(input_layer)
output_layer_position = Dense(1, name="regr", activation='sigmoid')(conv_model_position_flattened)

model = Model(inputs=input_layer, outputs=output_layer_position)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

In [ ]:
init_epoch = 0

In [ ]:

num_epochs = 10

def fit_model(epochs):
	
	global init_epoch
	history = model.fit(
		Train_Generator_RAW,
		steps_per_epoch = 1000,
		epochs = epochs + init_epoch,
		initial_epoch=init_epoch,
		verbose =1,
		# validation_data = Validation_Generator_RAW,
		# validation_steps = 1000,
		workers=12,
		use_multiprocessing=True
	)
					
	init_epoch += epochs
	return history

history = fit_model(num_epochs)


In [ ]:
# model = keras.models.load_model('saved_model_raw_binary')

In [ ]:
from multiprocessing import Pool


In [ ]:

def sum_up_to(generator):
    return generator.__getitem__(0)


def get_deviating_predictions(generator):

    a_pool = Pool()

    result = a_pool.map(sum_up_to, [generator for a in range(generator.__len__())])

    binary_differences = []

    for imgs_batch, labels_batch in result:

        predictions = model.predict_on_batch(imgs_batch) ## make the predictions before the loop, then insert predictions into multiprocessing functions

        for i in range(len(imgs_batch)):

            regr_pred = predictions[i]
            regr_test_label = labels_batch[i]

            if (round(regr_pred[0]) != round(regr_test_label)):
                binary_differences.append([regr_pred, regr_test_label, imgs_batch[i]])

    return binary_differences

In [ ]:
%%capture cap --no-stderr
from training_log import Training_Data_Log, print_name, json_to_file
import inspect

data_log = Training_Data_Log()
data_log.model_config = model.to_json()
data_log.model_config_method_string = [inspect.getsource(conv_model_velocity)]
# data_log.training_set_size = len(train)
# data_log.validation_set_size = len(train_validation)
# data_log.test_set_size = len(train_test)
data_log.image_pre_processors = print_name(IMAGE_PREPOCESSORS)
data_log.noise_added = ADD_NOISE_RANDOMLY
# data_log.training_data_masks = print_name(MASKS)
data_log.model_summary = model.summary()
data_log.model_optimizer = str(type(model.optimizer))
data_log.model_history = history.history
data_log.model_history_final_epoch = {k: v[-1] for k, v in history.history.items()}
data_log.total_epochs = init_epoch


In [ ]:
class Velocity_Results:
    total_predictions = None
    noise_level = None
    difference = None
    image_preprocessors_test = None
    predictions_off_by_more_than_difference = None
    predictions_incorrect_prercent = None
    model_evaluation = None

noise_levels = [0.0]
result_array = []
for noise_level in noise_levels:

    batches = 200

    # image_preprocessors_test = [{"func": add_noise, "params" : noise_level}]
    test_batch_generator = Train_Generator_RAW = Image_Generator_RAW(
        image_amount=BATCH_SIZE * batches,
        set_obj= set_obj,
        FFT_JUMP=64,
        batch_size=BATCH_SIZE,
        image_target_size=IMAGE_TARGET_SIZE,
        image_prepocessors=IMAGE_PREPOCESSORS,
        noise_range=ADD_NOISE_RANDOMLY,
        random_signal_indent=ADD_SIGNAL_INDENT_RANDOMLY,
        label_func = get_velocity_labels
    )
    # predictions = model.predict(test_batch_generator)
    # model.predict_on_batch

    binary_differences  = get_deviating_predictions(test_batch_generator)
    evaluations = model.evaluate(test_batch_generator, verbose = 0)

    results = Velocity_Results()
    results.total_predictions = batches * BATCH_SIZE
    results.noise_level = noise_level

    # results.image_preprocessors_test = print_name(image_preprocessors_test)
    results.predictions_off = len(binary_differences)
    results.predictions_incorrect_prercent = round( (  len(binary_differences) / (batches * BATCH_SIZE)  ) * 100, 4)
    results.model_evaluation = evaluations

    result_array.append(results.__dict__)


In [ ]:
data_log.results = result_array

In [ ]:
json_to_file("logs/binary/binary_data_log", data_log)

In [ ]:
# model.save("saved_model_raw_binary")

In [ ]:
for idx, diff in enumerate(binary_differences):

    if idx > 10:
        break

    pred, correct, img = diff

    print('Prediction', round(pred[0]))
    print('Correct', round(correct))
    show_image(img, 300)

    print("----------------------------------------------------------------------------------------")


In [ ]:
# model.save('saved_model_velocity_raw_23')